# 04: Decision trees
Decisions trees are simple yet powerful learning models. They partition the input space into many partitions: some with single examples, some with more. Their implementation can get a bit complex. Let's implement a decision tree a step by step. When done, we will pull everything together in a class. Here we are using the the ID3 (Iterative Dichotomiser 3) algorithm.

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mylib as my

## Setting up the dataset
We'll start by using the party example from the textbook.

In [2]:
df = pd.DataFrame({
    'deadline': ['Urgent', 'Urgent', 'Near', 'None', 'None', 'None', 'Near', 'Near', 'Near', 'Urgent'],
    'party': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'No'],
    'lazy': ['Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No'],
    'activity': ['Party', 'Study', 'Party', 'Party', 'Pub', 'Party', 'Study', 'TV', 'Party', 'Study']
})

ds = my.DataSet(df, y=True)

ds

  deadline party lazy      y
0   Urgent   Yes  Yes  Party
1   Urgent    No  Yes  Study
2     Near   Yes  Yes  Party
3     None   Yes   No  Party
4     None    No  Yes    Pub
5     None   Yes   No  Party
6     Near    No   No  Study
7     Near    No  Yes     TV
8     Near   Yes  Yes  Party
9   Urgent    No   No  Study

## Building the tree
The best way we know to build a decision tree is a greedy algorithm that builds the tree one feature (or node) at a time starting with the **best feature** at the moment. One way of specifying that is by quantifying the impurity of a node and selecting the feature that yields the least impurity. Here we have two such measures: **entropy** and **gini index**. 

### Using the entropy

Given a set of examples $D$ with labels $y_i$ where $i \in \{1, 2,\dots,L\}$ and $L$ is the number of unique labels, the entropy is defined as:

$${\displaystyle \mathrm {H(D)}=\sum _{i=1}^{L}{p_{i}\frac{1}{\log p_{i}}}=-\sum _{i=1}^{L}{p_{i}\log p_{i}}}$$

We use the entropy to calculate the information gain of a feature $F$ as follows:

$${\displaystyle \operatorname {Gain(F,D)} =H(D) -\sum_{f \in values(F)}\frac{|D_f|}{|D|}H(D_f)}$$

where $D_f$ is the set of all examples in $D$ where feature $F$ has the value $f$. Let's create function to calculate the information gain.


In [3]:
def entropy(df):
    p = df.iloc[:, -1].value_counts() / len(df)
    return (-p * np.log2(p)).sum()

def info_gain(df, feature):
    p = df[feature].value_counts() / len(df)
    
    for v in p.index:
        p.loc[v] *= entropy(df[df[feature] == v])
        
    return entropy(df) - p.sum()

Here are the gains for the three features of this dataset:

In [4]:
pd.Series({f: info_gain(df, f) for f in df.columns[:-1]})

deadline    0.534498
party       1.000000
lazy        0.209987
dtype: float64

Because the `party` feature has the largest gain, it becomes the root of the tree.

## Finding the best feature
This feature will be the with the maximum $Gain$.

In [5]:
def best_feature(df):
    features = df.iloc[:, :-1].columns
    info = pd.DataFrame({"feature": features})

    info['gain'] = [info_gain(df, f) for f in features]
    return info['feature'][info['gain'].argmax()]

For example, here is the best feature for to start the tree with.

In [6]:
best_feature(df)

'party'

## Constructing the tree
There are many ways to constructing the tree. Here, we'll use a Pandas Series object as a dictionary-like data structure to recursively (in a nested-way) store nodes of the tree. The first index of this Series object has the root node.

* First we find the best feature to add to the tree. The name of the feature will be the index and the value will be a Series object whose elements are either leaves (represented by tuples) or other series object for internal nodes.

* From the best feature, we get the all the unique values of the best feature. For each unique value:
    * Filter the data based on this value
    * Find the number of labels in the filtered dataset. 
    * If the number of labels is 1 then append a leaf as a child to this best feature
    * If the number of labels is > 1 then:
        * Find the next best feature using the filtered data
        * Repeat the process using the new best feature

Here is implementation of this algorithm:

In [7]:
tree = pd.Series(dtype=object)
def make_tree(df, node, feature=None):
    if feature is None:
        feature = best_feature(df)

    node[feature] = pd.Series(dtype=object)

    fvalues = df[feature].unique()
    for v in fvalues:
        d = df[df[feature] == v]
        n_classes = len(d.iloc[:, -1].unique())
        if n_classes == 1:
            node[feature][v] = ('L', d.iloc[:, -1].iloc[0])
        elif n_classes > 1:
            d = d.drop([feature], axis=1)
            if len(d.columns) == 1:
                node[feature][v] = ('L', d.iloc[:, -1].mode()[0])
            else:
                next_best_feature = best_feature(d)
                node[feature][v] = pd.Series(dtype=object)
                make_tree(d, node[feature][v], next_best_feature)

Here is a function to print the tree in an **inorder** manner:

In [8]:
def print_tree(name, tree, d=1):        
    for f in tree.index:
        if isinstance(tree[f], tuple):
            print('   ' * d, f, ' => ', tree[f], sep='')
        else:
            print('   ' * d, f, ': ', sep='')
            print_tree(f, tree[f], d + 1)

Here is the tree built using the `make_tree` function above.

In [9]:
make_tree(df, tree)
print_tree("", tree)

   party: 
      Yes => ('L', 'Party')
      No: 
         deadline: 
            Urgent => ('L', 'Study')
            None => ('L', 'Pub')
            Near: 
               lazy: 
                  No => ('L', 'Study')
                  Yes => ('L', 'TV')


The above tree representation corresponds to the following tree:

```
                              Feature:Party
                             /             \
                        Yes /               \ No
                           /                 \   
                      Leaf:Party       Feature:Deadline
                                      /       |        \
                              Urgent /        | None    \ Near
                                    /         |          \
                               Leaf:Study  Leaf:Pub       \
                                                           \
                                                      Feature:Lazy
                                                     /            \
                                                 No /              \ Yes
                                                   /                \
                                              Leaf:Study          Leaf:TV
```

## Using the tree
With a tree like this, how do we use it to predict the label of an unseen example? Well, it's a tree; so we use the unseen example to traverse the tree from the root until we reach a leaf. When we do reach a leaf, we report the majority label of all the examples under that leaf.

Here is a function to do that:

In [10]:
 def predict(unseen, node=None):
    """
    Returns the most probable label (or class) for each unseen input. 
    It can also be a single Series for a single example or a data frame 
    with many examples.
    """
    if unseen.ndim == 1:
        if node is None:
            node = tree
            
        feature = node.index[0]
        children = node[feature]
        value = unseen[feature]
        
        for c in children.index:
            if c == value:
                if isinstance(children[c], tuple):
                    return children[c][1]
                else:
                    return predict(unseen, children[c])
    else:
        return np.array([predict(unseen.iloc[i,:]) for i in range(len(unseen))])

Let's test it.

In [11]:
predict(pd.Series(['Near', 'No', 'Yes'], index=df.columns[:-1]))

'TV'

Here is the training confusion matrix and accuracy.

In [12]:
cm = my.confusion_matrix(df.iloc[:,-1].values, predict(df.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Training accuracy: ', accuracy)

[[5 0 0 0]
 [0 1 0 0]
 [0 0 3 0]
 [0 0 0 1]]
Training accuracy:  1.0


Wow! 100% accuracy. Is that a good thing, though?

## Putting things together
Let's put everything we did so far togeth into a class

In [13]:
class DecisionTreeClassifier:
    """ A basic ID3 Decision Tree"""

    def __init__(self, dataset):
        self.dataset = dataset
        self.tree = pd.Series(dtype=object)
        self.make_tree(self.dataset.examples, self.tree)
    
    def entropy(self, df):
        p = df.iloc[:, -1].value_counts() / len(df)
        return (-p * np.log2(p)).sum()

    def info_gain(self, df, feature):
        p = df[feature].value_counts() / len(df)

        for v in p.index:
            p.loc[v] *= self.entropy(df[df[feature] == v])

        return self.entropy(df) - p.sum()

    def best_feature(self, df):
        features = df.iloc[:, :-1].columns
        info = pd.DataFrame({"feature": features})
        info['gain'] = [self.info_gain(df, f) for f in features]
        return info['feature'][info['gain'].argmax()]
    

    def print_tree(self, name='', node=None, depth=1):
        if node is None:
            node = self.tree
            
        for f in node.index:
            if isinstance(node[f], tuple):
                print(' ' * depth, f, ' => ', tree[f], sep='')
            else:
                print(' ' * depth, f, ': ', sep='')
                print_tree(f, node[f], depth + 1)

    def make_tree(self, df, node, feature=None):
        if feature is None:
            feature = self.best_feature(df)

        node[feature] = pd.Series(dtype=object)

        fvalues = df[feature].unique()
        for v in fvalues:
            d = df[df[feature] == v]
            n_classes = len(d.iloc[:, -1].unique())
            if n_classes == 1:
                node[feature][v] = ('L', d.iloc[:, -1].iloc[0])
            elif n_classes > 1:
                d = d.drop([feature], axis=1)
                if len(d.columns) == 1: 
                    node[feature][v] = ('L', d.iloc[:, -1].mode()[0])
                else:
                    next_best_feature = self.best_feature(d)
                    node[feature][v] = pd.Series(dtype=object)
                    self.make_tree(d, node[feature][v] ,next_best_feature)
            else:
                pass

    def predict(self, unseen, node=None):
        """
        Returns the most probable label (or class) for each unseen input. 
        It can also be a single Series for a single example or a data frame 
        with many examples.
        """
        if unseen.ndim == 1:
            if node is None:
                node = self.tree

            feature = node.index[0]
            children = node[feature]
            value = unseen[feature]
            for c in children.index:
                if c == value:
                    if isinstance(children[c], tuple):
                        return children[c][1]
                    else:
                        return self.predict(unseen, children[c])
        else:
            return np.array([self.predict(unseen.iloc[i,:]) for i in range(len(unseen))]) 
    
    def __repr__(self):
        return repr(self.tree)
    

Let's test if by taking the party dataset and dividing it into a three-example test set and a seven-example training set.

In [14]:
train, test = ds.train_test_split(start=0, end=3, shuffle=True)
print("Training set: \n", train)
print("\nTest set: \n", test)

Training set: 
   deadline party lazy      y
0   Urgent   Yes  Yes  Party
6     Near    No   No  Study
1   Urgent    No  Yes  Study
2     Near   Yes  Yes  Party
4     None    No  Yes    Pub
5     None   Yes   No  Party
8     Near   Yes  Yes  Party

Test set: 
   deadline party lazy      y
7     Near    No  Yes     TV
3     None   Yes   No  Party
9   Urgent    No   No  Study


In [15]:
dt = DecisionTreeClassifier(train)
print(dt.predict(pd.Series(['Near', 'No', 'Yes'], index=df.columns[:-1])))

cm = my.confusion_matrix(train.examples.iloc[:,-1].values, dt.predict(train.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)
print(cm)
print('Training accuracy: ', accuracy)

cm = my.confusion_matrix(test.examples.iloc[:,-1].values, dt.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)
print(cm)
print('Test accuracy: ', accuracy)

Study
[[4 0 0]
 [0 1 0]
 [0 0 2]]
Training accuracy:  1.0
[[1 0 0]
 [0 1 0]
 [0 1 0]]
Test accuracy:  0.6666666666666666


Keep in mind that you might get an error when you test, simply because the tree might not return a label.

## CHALLENGE
**PART 1**: Test the class above using the `datasets/restaurant.csv` dataset. Report both the training and test accuracy.

**PART 2**: If instead of the information gain:
$${\displaystyle \operatorname {Gain(F,D)} =H(D) -\sum_{f \in values(F)}\frac{|D_f|}{|D|}H(D_f)}$$

we use the weighted entropy:
$${\displaystyle \operatorname {Entropy(F,D)} = \sum_{f \in values(F)}\frac{|D_f|}{|D|}H(D_f)}$$

to determine the best feature. Does that work? Try answering that by 
* removing the `info_gain` method of the above class and replacing it with `weighted_entropy` as described above. Keep in mind that in this case, the best feature is the one with the minimum weighted entropy.
* testing the modified class using the same restaurant training and test sets from PART 1.